In [1]:
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [6]:
%matplotlib

Using matplotlib backend: <object object at 0x0000021FD8774A50>


In [14]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots()

figraw,axsraw = plt.subplots()

figz,axz = plt.subplots()


fig.suptitle('Individual RL Tuned Responses for Squash Ball',fontsize=15)
figraw.suptitle('Unfiltered RL Tuned Responses for Squash Ball',fontsize=15)
figz.suptitle('Motor Angle for RL Tuned Squash Ball Response',fontsize=15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of RL Tuned Squash Ball Responses',fontsize=15)
#fig3.suptitle('Unfiltered Average of Trial-and-Error Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]',fontsize=15)
ax2.set_ylabel('Distance [cm]',fontsize=15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 185
for i in range(1,8):
    df = pd.read_csv('RL_Test_{}.csv'.format(i))
    
    time = (df['time'])/1000-5
    print(len(time))
    time = time[0:END]
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(time,filtered,label = 'Filtered {}'.format(i))
    #axs.plot(time,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs.set_xlabel('Time [s]',fontsize=15)
    axs.set_ylabel('Distance [cm]',fontsize=15)

    
    
    axsraw.plot(time,distance)
    axsraw.set_xlabel('Time [s]')
    axsraw.set_ylabel('Distance [cm]')
    axsraw.grid()
    
    axz.plot(time,angle_filt,label = 'Filtered {}'.format(i))
    #axz.plot(time,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz.set_xlabel('Time [s]',fontsize=15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize=15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
axs.grid()    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axs.legend()
axz.legend()
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)

filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
OS = np.max(filtered)
OSind = np.argmax(filtered)
OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax2.axhline(filtered[-1]*1.05,color = 'g',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid(which='both')
ax2.plot(sett,Usett,'p',label = "settling point",color = 'r')
ax2.axhline(filtered[-1],color='b',label = "steady state value",alpha=0.6,linestyle = 'dotted')
ax2.vlines(OST,15,OS,color = 'r',label = 'steady state error')
ticks = np.linspace(0,18,19)
ax2.set_yticks(ticks)
ax2.legend()
#ax3.plot(time_ave,dist_ave,label = 'response')
print('Overshoot=',((np.max(filtered)-filtered[-1])/filtered[-1])*100)
print('Settle =',(sett))
print("SSE=", (filtered[-1]-15)/0.15)

494
270
614
274
382
185
227
Overshoot= 5.960654639980131
Settle = 11.0442
SSE= -11.498865353371162
